# Chapter 20 - Statistical Significance in Linguistics

In this chapter, I will provide a very basic overview of statistical significance
tests for linguistic entities. Continuing from Chapter 19, we've defined a linguistic
hypothesis as the following:

    a "statement postulating relationships between
    [two or more] constructs". (Stefanowitsch 2020: 64)
    
We also pointed out that the primary means of testing linguistic hypotheses is
with statistical methods. 

Statistics possesses a powerful toolkit for testing hypotheses regarding relationships
between two categories. These tools are necessary for determining, for instance, how
effective a given medicine is on a test population. In medical trials, there is typically
two groups: one which receives the drug being tested, and a control group which receives
a placebo drug. Data is gathered on how each group responds to certain conditions, such as 
whether the drug improves a person's symptoms. If the group who receives the drug shows
improvement above a certain threshold, statistical significance, then the study can reject
the null hypothesis:

    the null hypothesis is that there is no significant 
    relationship between two or more categories
    
Thus, if the trial group shows improvement above a threshold of statistical significance,
the researchers can reject the null hypothesis. In other words, they cannot disprove 
that there *is* an effect caused by the drug. 

**Important: in science we aim to falsify claims, not to prove them**. This is key. In the
drug study, it would be irresponsible for the researchers to immediately claim their results
prove a drug's effectiveness before their results have been replicated by others. That is because
there are always other potential factors involved, and the study involves only a subset of an
entire population. If a sufficient number of other researchers cannot falsify the results with
other similar trials, then the scientific community can tentatively adopt the hypothesis
that there is a positive effect caused by the drug.

**Important: correlation is not causation**. This phrase is nearly a cliché by now. But it is
especially important to keep in mind when working with linguistic data. Language is a very 
interconnected and complex network of patterns and meanings that interact in all sorts of ways.
You should always take care when formulating claims based on statistical data.

**Important: statistical data is always *interpreted***. Statistical results must, in the end,
make sense of your dataset. I like how Stefanowitsch puts it, regarding the difference between
intuitional data (such as grammaticality judgments) and interpretive data (such as reading
statistical results): 

> We need to distinguish two different kinds of introspection: 
> (i) *intuiting*, i.e. the practice of introspectively accessing 
> one's linguistic experience in order to create sentences and assign
> grammaticality judgments to them and (ii) *interpreting*, i.e. the
> practice of assigning an interpretation (in semantic and pragmatic terms)
> to and utterance. (Stefanowitsch, *Corpus Linguistics*, 2020, 8).

Indeed, we can extend this reasoning to the interpretation of statistical data. Statistical methods
are not simply a kind of machine that produce *results*. Even unsupervised methods
do not produce results on their own. True results are interpreted against the backdrop
of data. However, even though we rely on our subjective interpretation, we are in a 
better position to be guided by the data than by creating our own data to analyze (sort of
like grading your own homework).

**Important: these chapters are not a substitute for a proper introduction to statistics.** No really.
There are a lot of important concepts that I will not cover here for the sake of time and space. 
The concept of normal distributions, for example, is extremely important for statistics. But since
much language data is not normally distributed, we will not discuss that here. Nevertheless, you
should consult real statistical guides before applying these own methods to your own research. For this,
I highly recommend either Stefanowitsch, Gries, or Levshina:

* [Anatol Stafanowitsch, *Corpus Linguistics: A Guide to Methodology*, 2020.](https://langsci-press.org/catalog/book/148)
* [Stefan Gries, *Statistics for Linguistics with R*, 2013.](https://www.degruyter.com/view/title/300775)
* [Natalia Levshina, *How to do Linguistics with  R: Data Exploration and Statistical Analysis*, 2015.](https://benjamins.com/catalog/z.195)

## Why are raw counts not sufficient?

Please [read this article](https://www.researchgate.net/publication/290329368_Quantitative_designs_and_statistical_techniques) 
to see why you should avoid using only raw counts in your linguistic research.

## Mock Dataset from the Hebrew Bible

Below we construct a new dataset that we'll refer to throughout this chapter. 
That dataset is constructed using the [corpus analysis tool Text-Fabric](https://annotation.github.io/text-fabric/)
using the linguistic annotations provided by the [ETCBC](http://github.com/etcbc/bhsa). If you don't have Text-Fabric
installed already, go ahead and run the first cell.

Here is our hypothesis that we will test:

    There is a relationship between discourse
    type in the Hebrew Bible and word order in
    the clause
    
Here is the operationalizations we make to test this hypothesis:

* discourse type - "N" for narrative, "Q" for quotation
* word order - "SV" for subject before verb, "VS" for verb before subject
* clause - a clause as delineated in the BHSA by the ETCBC Amsterdam; only clauses with a subject and verb
* Hebrew Bible - BHS linguistically encoded by the ETCBC Amsterdam

In [ ]:
# If you don't have Text-Fabric installed,
# uncomment below and run

#! pip install text-fabric

In [ ]:
# load modules for analysis
import pandas as pd
import pprint # for pretty printing dictionaries
import matplotlib.pyplot as plt

In [ ]:
# load the BHSA dataset
from tf.app import use

bhsa = use('bhsa') # downloads and loads the HB data — may take some time to complete

Ignore for the moment the messages about the rate limits.

Our data has loaded. When working with Text-Fabric, there are a set of classes and methods that we 
regularly use to navigate a corpus. I assign those classes/methods now to a 
set of short-form variables.

In [ ]:
F = bhsa.api.F # for features of nodes
L = bhsa.api.L # for moving through contexts
T = bhsa.api.T # for accessing convenient text/reference data

With these two variables we can interate through the whole BHSA dataset
and collect our data. We will place it into a **raw data table** as is
standard.

We iterate through every clause in the Hebrew Bible. If the clause has 
a subject and a verb, we store that clause along with the relevant data.
We include reference data so we can locate it later if need be.

In [ ]:
bhsa_raw_data = []

# iterate through the necessary items in BHSA
# in Text-Fabric, each linguistic unit is converted
# to an integer that can be used to look up relevant
# linguistic/contextual data. We use `F` and `L` to 
# do that. `F` looks up features, whereas `L` looks
# up other units within a supplied context

for clause in F.otype.s('clause'): # iterate through every clause in HB
    
    word_order = '' # store order info here
    phrases = {} # store phrase nodes here

    # word order string is added to as 
    # constituents are encountered; the 
    # constituents themselves are saved in
    # `phrases` dict for later reference
    for phrase in L.d(clause,'phrase'):
        function = F.function.v(phrase)
        if function == 'Subj':
            word_order += 'S'
            phrases['S'] = phrase
        elif function == 'Pred':
            word_order += 'V'
            phrases['V'] = phrase
            
    # retrieve the discourse type of the clause
    discourse_type = F.txt.v(clause)
            
    # now we make a bunch of qualifications about 
    # which clauses we should keep and which we should skip
        
    # skip clause if there is no subject or verb 
    # we'll use sets to check for both values
    # if there is anything lacking from set {'S','V'}
    # then the clause does not qualify!
    if {'S','V'} - set(phrases.keys()):
        continue
    
    # skip if too many phrases matched (i.e. multiple constituents)
    if len(phrases) > 2:
        continue
        
    # skip clause if the discourse type is not N or Q
    if discourse_type not in {'N', 'Q'}:
        continue
        
    # assemble some English glosses for phrases
    kind2gloss = {}
    for kind,phrase in phrases.items():
        glosses = []
        for word in L.d(phrase,'word'):
            glosses.append(F.gloss.v(word))    
        glossed = ' '.join(glosses)
        kind2gloss[kind] = glossed
        
    # now we assemble the rest of the data we want to store
    book,chapter,verse = T.sectionFromNode(clause)
    reference = f'{book} {chapter}:{verse}'
    subj_phrase_gloss = kind2gloss['S']
    verb_phrase_gloss = kind2gloss['V']
    clause_text = T.text(clause)
    
    # build up clause data as a big dict
    clause_data = {
        'reference': reference,
        'book': book,
        'clause_node': clause,
        'text': clause_text,
        'word_order': word_order,
        'discourse_type': discourse_type,
        'subj_phrase': phrases['S'],
        'verb_phrase': phrases['V'],
        'subj_glossed': kind2gloss['S'],
        'verb_glossed': kind2gloss['V'],
    }
    
    # add data to whole dataset
    bhsa_raw_data.append(clause_data)
    
# report on our data
print(len(bhsa_raw_data), 'clauses gathered')
print('sample:')
pprint.pprint(bhsa_raw_data[:3], sort_dicts=False) # pprint will print dicts with nice indentation

Now we arrange the data into a DataFrame for analysis. Note that since we used
a dictionary for each row, we do not have to specify column labels. They are inferred
from the data by Pandas!

In [ ]:
clause_df = pd.DataFrame(bhsa_raw_data)

clause_df.head()

## Data Exploration

Before you begin to analyze a dataset, you should always start by exploring your dataset.
This ensures that your data is arranged as expected.

A particularly useful method with a DataFrames is `value_counts`:

In [ ]:
clause_df['word_order'].value_counts().plot(kind='bar')

In [ ]:
clause_df['discourse_type'].value_counts().plot(kind='bar')

We see that we have a lot more of `VS` and `N` types than `SV` and `Q`. Remember our hypothesis:

    There is a relationship between discourse
    type in the Hebrew Bible and word order in
    the clause

This immediately raises the question: How do we ensure that our comparisons will 
be "fair"? Since `VS` and `N` occur so much more frequently, we might find it 
difficult to reach a sound conclusion. If `VS` occurs so much more in `N`, it 
may not be because there is a meaningful relationship there, it could just be 
due to sample sizes.

Let's try to look at proportions first. We will need a summary table of how often
each of these respective values co-occur. We will use the Pandas method called
the [pivot table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) 
to cross-tabulate the counts of the values together.

Note that because we are interested in whether discourse type affects word order,
discourse type is our independent variable. To put it another way, we hypothesize
that word order is dependent on discourse type. 

**Remember that we put independent variables in rows (index in DataFrame) and 
dependent variables in columns**. You will see these values reflected in the
`pivot_table` below. The method (`aggfunc`) we use for counting occurrences is called
`size`. It is one of several count methods you can supply to a pivot table.

In [ ]:
discourse_order = pd.pivot_table(clause_df, index='discourse_type', columns='word_order', aggfunc='size')

discourse_order

Now we have the raw co-occurrence frequencies for these values, calculated directly
from our raw data. We can convert this to a proportion across discourse type. That is,
for each discourse type, we calculate what percentage is represented by SV or VS.

We can use Pandas `.divide` and `.sum` to specify how to add and divide each row/column.
In Pandas a `0` is the "index" and a `1` is a column.

In [ ]:
discourse_proportion = discourse_order.divide(discourse_order.sum(1), 0)

discourse_proportion

We know we've done it correctly because the sum of N and Q add up to 1 (i.e. 100%):

In [ ]:
discourse_proportion.sum(1) # sum across columns (i.e. for each row)

We can see that 10% of narrative discourse occur with SV word order, for instance.

But notice what we've done: we are looking at what proportion of discourse type is
represented by a certain word order. We can also look at that from the other perspective:
what proportion of a certain word order is represented by a discourse type? In other words,
there is a bi-directional nature of proportionality here, represented across all 4 possible
combinations.

Below, we calculate the proportions across word order.

In [ ]:
order_proportion = discourse_order.divide(discourse_order.sum(0), 1)

order_proportion

In [ ]:
order_proportion.sum(0)

**Notice that a higher proportion of SV occurs with N than does N with SV!** In the discourse proportions
we saw that SV accounts for 10% of all of N's occurrences. But here we see that
N accounts for 35% of SV's occurrences. That's a substantial gap!

This is why we need **contingency data**. Contingency data takes into account
all values. Those values can then be used to calculate various measures of 
statistical significance.

The statistical significance will then allow us to answer our question about the relationship
between word order and discourse type.

## The contingency table

The typical form of a contingency table for language data
is the following (see Levshina 224 for more):

|         | feature | ¬feature |
|---------|---------|----------|
| sample  |    A    |     B    |
| ¬sample |    C    |     D    |

Where sample is a independent entity, feature is the dependent
entity, and where ¬ means "not", i.e. ¬feature means every other
feature besides the first feature. 

The values A, B, C, D are sums of those respective cross tabulations.
So A is the sum of the co-occurrence of sample x feature. B is the sum
of a sample x all other features, etc.

Given a sample and a feature count in a dataset, the math 
for finding A, B, C, D (see Levshina 2015, 223) is:

```
        A = frequency of sample w/ feature (in dataset)
        B = sum(sample) - A
        C = sum(feature) - A
        D = sum(dataset) - (A+B+C)
```

What this means for our dataset above is that each one of the 4 values 
[i.e. (N,D) x (SV,VS)] has its own contingency data. That means we need
to calculate the values independently for each one and assemble them into a new
form.

There are more efficient and less efficient ways to do this. But the simplest
way, when dealing with a relatively small dataset, is to do a double for loop. 
We do that below and re-assemble the data into 4 different tables each corresponding
to a, b, c, and d and store them in a dict. Then, the values of each of the 4 
tables will line up with the original sample x feature pairs. If this is confusing,
try to follow the code below.

In [ ]:
def contingency_tables(counts_df):
    """Calculate contingency tables for a DF of raw counts"""
    
    # each table assembled here
    # the list will become a list of row lists
    cont_values = {
        'a': [],
        'b': [],
        'c': [],
        'd': [],
    }

    # iterate through raw data table and 
    # assemble a,b,c,d matrices
    for sample in discourse_order.index:

        # new rows of a,b,c,d assembled here
        a_row = []
        b_row = []
        c_row = []
        d_row = []

        # calculate the contigency scores 
        for feature in discourse_order.columns:
            a = counts_df.loc[sample][feature]
            b = counts_df.loc[sample].sum() - a
            c = counts_df[feature].sum() - a
            d = counts_df.sum().sum() - (a+b+c) # .sum().sum() to get whole dataset sum

            # stash in new rows
            a_row.append(a) 
            b_row.append(b)
            c_row.append(c)
            d_row.append(d)

        # add new row lists to table list
        cont_values['a'].append(a_row)
        cont_values['b'].append(b_row)
        cont_values['c'].append(c_row)
        cont_values['d'].append(d_row)
  
    # return the dictionary
    return cont_values

Now we apply the new function to our dataset below.

In [ ]:
cont_values = contingency_tables(discourse_order)

In [ ]:
pprint.pprint(cont_values)

Each dictionary value can be converted back to a DataFrame so that we can work with it. We do that below.
Note that we must re-specify the name of the indices and columns.

In [ ]:
for value, table in cont_values.items():
    cont_values[value] = pd.DataFrame(table, index=discourse_order.index, columns=discourse_order.columns)

Now we can refer to each dataframe directly!

In [ ]:
cont_values['a'].head()

Note that a is the same as a raw dataframe. That is expected. But look at b:

In [ ]:
cont_values['b']

For the sake of succinctness, we assign each value now to its own variable:

In [ ]:
# make some short-hand names for clearer code
a = cont_values['a']
b = cont_values['b']
c = cont_values['c']
d = cont_values['d']

## Statistical Association with Fisher's Exact

The Fisher's Exact test is a powerful test for statistical significance. Fisher's returns
a value, called the `p-value`, which is an important measure in statistics of significance.

**The p-value tells us the likelihood that two variables are 
would co-occur with a given frequency by chance.** Thus, the lower 
the p-value, the more likely there is a significant and meaningful
relationship. 

**By convention, a p-value < 0.05 is accepted as statistically significant**. This is 
simply a standard often observed in statistics (though not always). A p-value < 0.05
means that there is a less than 5% chance that the observed frequency is due to random
chance. At that threshold, we are safe to reject the null hypothesis, that there is no relationship.

Fisher's Exact can be calculated using the `scipy` stats module. We also import numpy as a helper tool.

In [ ]:
import numpy as np
import scipy.stats as stats

Have a look at the Fisher's test requirements:

In [ ]:
help(stats.fisher_exact)

Note that the test takes a 2x2 contingency table. 

We will now build a DataFrame with the Fisher's test. We use feed the test
the requested contingency data as we iterate through the dataset. We assemble
the new data into a new dataframe arranged similarly to the old dataframe.

In [ ]:
fishers_data = []

for sample in discourse_order.index:
    new_row = []
    for feature in discourse_order.columns:
        
        # use numpy to form new matrix
        contingency = np.matrix(
            [
                [a[feature][sample], b[feature][sample]], 
                [c[feature][sample], d[feature][sample]]
            ]
        )
        oddsratio, p_value = stats.fisher_exact(contingency)
        new_row.append(p_value)
    fishers_data.append(new_row)
    
# make new DF
fishers_data = pd.DataFrame(fishers_data, index=a.index, columns=a.columns)

In [ ]:
fishers_data

Note that the answer is given to us with scientific notation. These numbers
are so small, over 266 decimal places, that they are nearly 0. That means that
according to the Fisher's test, there is a near zero chance each of these 4
values is due to randomness.

**Based on the Fisher's Exact analysis, we can reject the null hypothesis that 
discourse type has no effect on word order in the Hebrew Bible.**

## Statistical Association with ΔP

We are finally in a position to posit an answer for our research question. For this,
we need an appropriate measure of statistical significance. ΔP is such a measure, designed
for testing hypotheses in Psycholinguistics. 

It is worth reading about ΔP separately (see article in the Slack channel), because it is a powerful tool for measuring
the relationships of linguistic items. Here's a few points about ΔP:

* based on the concept of a cognitive "cue" and "response", where a "cue" is the independent value, and the "response" is the dependent value
* not based on normal distribution assumptions
* returns a value from 1 to -1, where 1 is the strongest association, and -1 is the strongest *dis*association.

ΔP can be calculated with the data we've just organized. Here's the formula for it:

$\frac{a}{(a+b)} - \frac{c}{(c+d)}$

Where a, b, c, and d are the contigency values we've already calculated.

This represents the probability of an observed collocation
(C) given a target construction (CX) minus the probability 
of the observed collocation without the target construction 
(adapted from Ellis 2006: 11):

$P(C|CX) - P(C|-CX)$

If the math is weird to you, just follow the Python code below!

In [ ]:
# calculate DP for our dataset
# note we use the parentheses in Python
# to tell it order of operations, this prevents
# b - c, for example
# note also that a, b, c, d are each full-fledged DataFrames!
# thus we calculate DP for the whole dataset at once

dp_df = a/(a+b) - c/(c+d)

Now let's have a look at our table. We can read the table thusly:

What is probabilty that the item in row, cues or triggers, the item in the column. 

In [ ]:
dp_df

**Here are the final results!** We can read this as: 

* If we are in narrative, we are 30% less likely to see SV, and 30% more likely to see VS
* If we are in quotation, we are 30% more likely to see SV, and 30% less likely to see VS

**Based on the ΔP analysis, we can reject the null hypothesis that 
discourse type has no effect on word order in the Hebrew Bible.**